In [2]:
import pickle
import numpy as np 

from sklearn.model_selection import train_test_split

import keras
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten
from keras.applications.resnet50 import ResNet50

Using TensorFlow backend.


In [2]:
pristine_data = np.load('sample_pristine_np.npy')

pristine_data.shape

(47150, 64, 64, 3)

In [25]:
fake_data = np.load('sample_fakes_np.npy')

fake_data.shape

(46235, 64, 64, 3)

In [3]:
from keras.models import load_model

model = load_model('best_model(1).h5')
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 131201    
Total params: 14,845,889
Trainable params: 2,491,009
Non-trainable params: 12,354,880
_________________________________________________________________


In [5]:
feature = model.layers[0].predict(pristine_data)

In [6]:
feature.shape

(47150, 2, 2, 512)

In [7]:
feature_flattened = feature.reshape(-1, 2*2*512)

In [8]:
feature_flattened.shape

(47150, 2048)

In [58]:
feature_flattened_pristine = feature_flattened

In [26]:
fake_feature = model.layers[0].predict(fake_data)

In [59]:
feature_flattened_fake = fake_feature.reshape(-1, 2*2*512)

In [3]:
with open('vgg16_stride16_fake_features.pickle', 'rb') as f:
    feature_flattened_fake = pickle.load(f)
    
with open('vgg16_stride16_pristine_features.pickle', 'rb') as f:
    feature_flattened_pristine = pickle.load(f)

Let's first try One-Class SVM using VGG16 features directly without compression

In [4]:
from sklearn import svm

model = svm.OneClassSVM(kernel='rbf', nu=0.0005,gamma=0.007)
model.fit(feature_flattened_pristine)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.007, kernel='rbf',
            max_iter=-1, nu=0.0005, random_state=None, shrinking=True,
            tol=0.001, verbose=False)

In [5]:
y_train_pred = model.predict(feature_flattened_pristine)

sum(y_train_pred == 1 ), len(y_train_pred), sum(y_train_pred == 1 )/len(y_train_pred)

(18535, 47150, 0.3931071049840933)

In [7]:
feature_flattened_pristine.shape, feature_flattened_fake.shape

((47150, 2048), (46235, 2, 2, 512))

In [8]:
y_test_pred = model.predict(feature_flattened_fake.reshape(-1, 2*2*512))

sum(y_test_pred == 1 ), len(y_test_pred), sum(y_test_pred == 1 )/len(y_test_pred)

(0, 46235, 0.0)

In [27]:
"""
with open("vgg16_stride16_fake_features.pickle", "wb") as f:
    pickle.dump(fake_feature, f)
    
"""

In [21]:
from keras.layers import Input, Dense
from keras.models import Model


num_features = feature_flattened.shape[1]

# this is our input placeholder
input_img = Input(shape=(num_features,))

# "encoded" is the encoded representation of the input
encoded = Dense(512, activation='relu')(input_img)
encoded = Dense(100, activation='relu')(encoded)
#encoded = Dense(2000, activation='relu')(encoded)


encoded = Dense(50, activation='sigmoid')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(100, activation='relu')(encoded)
# decoded = Dense(2000, activation='relu')(decoded)
decoded = Dense(512, activation='relu')(decoded)
decoded = Dense(num_features)(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)
encoder = Model(input_img, encoded)

autoencoder.compile(optimizer='adam', loss='mse')
#estop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

In [14]:
x_train, x_cv = train_test_split(feature_flattened, train_size = 0.8, shuffle = True)

x_train.shape, x_cv.shape

((37720, 2048), (9430, 2048))

In [23]:
from keras.callbacks import TensorBoard

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import matplotlib.pyplot as plt 

earlystopper = EarlyStopping(patience=6, verbose=1) 
# checkpointer = ModelCheckpoint('autoencoder.h5', verbose=1, save_best_only=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

class ImageSaver(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        pass
 
    def on_epoch_begin(self, epoch, logs={}):
        if epoch % 5 == 0:
            #encoded_imgs = encoder.predict(X_cv_flattened[0:2])
            #decoded_imgs = decoder.predict(encoded_imgs)
            decoded_imgs = autoencoder.predict(X_cv_flattened[0:2])
            
            plt.imsave('img/track_1_%d.png' % (epoch), decoded_imgs[0].reshape(64, 64, 3))
            plt.imsave('img/track_2_%d.png' % (epoch), decoded_imgs[1].reshape(64, 64, 3))
            
        #logging.info("Epoch: {}".format(epoch))

        
autoencoder.fit(x_train, x_train,
                epochs=500,
                batch_size=2048,
                validation_data=(x_cv, x_cv),
                callbacks=[earlystopper])
               

Train on 37720 samples, validate on 9430 samples
Epoch 1/500
37720/37720 [==============================] - 4s 107us/step - loss: 26.7321 - val_loss: 29.9329
Epoch 2/500
37720/37720 [==============================] - 4s 95us/step - loss: 26.7224 - val_loss: 29.9501
Epoch 3/500
37720/37720 [==============================] - 4s 105us/step - loss: 26.6834 - val_loss: 29.9336
Epoch 4/500
37720/37720 [==============================] - 4s 93us/step - loss: 26.6692 - val_loss: 29.8987
Epoch 5/500
37720/37720 [==============================] - 3s 92us/step - loss: 26.6628 - val_loss: 29.8863
Epoch 6/500
37720/37720 [==============================] - 3s 91us/step - loss: 26.6414 - val_loss: 29.8876
Epoch 7/500
37720/37720 [==============================] - 3s 88us/step - loss: 26.5885 - val_loss: 29.8430
Epoch 8/500
37720/37720 [==============================] - 3s 81us/step - loss: 26.5651 - val_loss: 29.8542
Epoch 9/500
37720/37720 [==============================] - 3s 79us/step - loss: 26.55

In [30]:
autoencoder.save("AE_using_feature_vgg16.h5")

Using features extracted from VGG16, and then compressed using Auto Encoder to dim = 50 

In [70]:
AE_train_feature = encoder.predict(feature_flattened_pristine)
AE_fake_feature = encoder.predict(feature_flattened_fake)

In [71]:
AE_train_feature.shape, AE_fake_feature.shape

((47150, 50), (46235, 50))

In [62]:
from sklearn import svm

model = svm.OneClassSVM(kernel='rbf', nu=0.0005,gamma=0.007)
model.fit(AE_train_feature)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.007, kernel='rbf',
            max_iter=-1, nu=0.0005, random_state=None, shrinking=True,
            tol=0.001, verbose=False)

In [63]:
y_train_pred = model.predict(AE_train_feature)

In [66]:
sum(y_train_pred == 1 ), len(y_train_pred), sum(y_train_pred == 1 )/len(y_train_pred)

(47125, 47150, 0.9994697773064687)

In [72]:
y_test_pred = model.predict(AE_fake_feature)

In [74]:
sum(y_test_pred == 1 ), len(y_test_pred), sum(y_test_pred == 1 )/len(y_test_pred)

(46227, 46235, 0.9998269709094841)

Using features extracted from VGG16, and then compressed using PCA to retain 75% of variance

In [28]:
from sklearn.decomposition import PCA 

In [39]:
pca = PCA(n_components=256)
pca.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=256, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [40]:
sum(pca.explained_variance_ratio_)

0.7403305729385465

In [41]:
x_train_transformed =pca.transform(x_train)
x_cv_transformed = pca.transform(x_cv)

x_train_transformed.shape, x_cv_transformed.shape

((37720, 256), (9430, 256))

x_train, x_cv contains only pristine (label = 0) samples 

In [42]:
from sklearn import svm

In [43]:
model = svm.OneClassSVM(kernel='rbf', nu=0.0005,gamma=0.007)
model.fit(x_train_transformed)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.007, kernel='rbf',
            max_iter=-1, nu=0.0005, random_state=None, shrinking=True,
            tol=0.001, verbose=False)

In [45]:
y_train_pred = model.predict(x_train_transformed)

In [46]:
y_train_pred

array([1, 1, 1, ..., 1, 1, 1])

In [51]:
sum(y_train_pred == 1), sum(y_train_pred == -1)

(18577, 19143)

Let's now make some tampered samples to test, and name is x_test 

EDIT: turns out there's no point because the percentage is almost 50: 50 

In [ ]:
x_test = fake_data[:50]
x_test_transformed = pca.transform(x_test)

In [ ]:
y_test_pred = model.predict(x_test_transformed)